In [1]:
import numpy as np

In [3]:
def separate_data(data):
    np.random.shuffle(data)
    n = data.shape[0]
    m = data.shape[1]
    
    # separate into training and test sets
    training_data = data[0:int(0.9*n),]
    test_data = data[int(0.9*n):n,]
    
    # y is first column, convert units
    y_training = training_data[:,0]*1000.0
    y_test = test_data[:,0]*1000.0
    
    # x is everything but the first and last columns
    X_training = training_data[:,1:m-1]
    X_test = test_data[:,1:m-1]
    
    # normalize features
    #X_training, X_test = normalize_features(X_training, X_test)
    
    # add bias feature
    X_training = np.append(X_training, np.ones([int(0.9*n),1]), 1)
    X_test = np.append(X_test, np.ones([n-int(0.9*n),1]), 1)

    return y_training, y_test, X_training, X_test

In [2]:
def normalize_features(training_data, test_data):
    m = np.mean(training_data, 0) 
    s = np.std(training_data, 0)
    s[s == 0] = 1
    training_normalized = (training_data - m) / s
    test_normalized = (test_data - m) / s
    return training_normalized, test_normalized

In [ ]:
def remove_bad_data(data):
    ind = data['Event type'] == 'None' & data['Observation count'] == 24
    data = data[ind]
    return

In [4]:
a = np.array([[1.0, 2.0, 3.0],[2,3,4]])
np.mean(a,0)
a

array([[ 1.,  2.,  3.],
       [ 2.,  3.,  4.]])

In [1]:
def add_gc(AQS, GCsites, GC1, GC2, GC3, GC4, GC5, GC6, GC7, GC8, GC9): 
    # give GC unique ids
    GCsites = create_unique_ids(GCsites, datatype='GC')
    
    # get unique ids for AQS data
    AQS = create_unique_ids(AQS, datatype='AQS')
    
    # empty array for GC_MDA8
    GC_MDA8 = np.zeros(AQS.shape[0])
    mon = np.zeros(AQS.shape[0])
    
    # loop through all AQS entries and add corresponding GEOS-Chem entry
    for n in xrange(0,AQS.shape[0]):
        aqsID = AQS['uniqueID'].values[n]
        date = AQS['Date Local'].values[n]
        month = np.int(date[5:7]) 
        day = np.int(date[8:10])
        i = day - 1
        j = np.where(gcsites.uniqueID == int(aqsID))
        if len(j[0]) > 0:
            mon[n] = month
            if (month == 1): 
                GC_MDA8[n] = GC1[i,j]
            elif (month == 2):
                GC_MDA8[n] = GC2[i,j]
            elif (month == 3):
                GC_MDA8[n] = GC3[i,j]
            elif (month == 4):
                GC_MDA8[n] = GC4[i,j]
            elif (month == 5):
                GC_MDA8[n] = GC5[i,j]
            elif (month == 6): 
                GC_MDA8[n] = GC6[i,j]
            elif (month == 7):
                GC_MDA8[n] = GC7[i,j]
            elif (month == 8):
                GC_MDA8[n] = GC8[i,j]
            else:
                GC_MDA8[n] = GC9[i,j]
        
    AQS['GC'] = GC_MDA8
    AQS['month'] = mon
    return AQS

In [6]:
def create_unique_ids(data, datatype):
    # loop through data and add new column of unique ids
    if (datatype == 'GC'):
        data['uniqueID'] = get_unique_id(data['SITE_ID'].values, 
                                         data['STATE'].values, data['LATITUDE'].values)
    else:
        data['uniqueID'] = get_unique_id(data['Site Num'].values, 
                                         data['State Name'].values, data['Latitude'].values)
    return data

In [7]:
def get_unique_id(SiteID, StateName, Lat):
    stateID = get_state_id(StateName)
    latID = (Lat*1000).astype(int) % (Lat.astype(int)*1000)
    uniqueID = (stateID*10000000.0) + (SiteID*1000.0) + latID
    return uniqueID

In [8]:
# returns the state ID given the state name
def get_state_id(State_Name):
    state_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                  'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 
                  'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
                  'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 
                  'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 
                  'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 
                  'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 
                  'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
                  'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
                  'West Virginia', 'Wisconsin' ,'Wyoming', 'District Of Columbia', 
                  'Puerto Rico', 'Country Of Mexico']
    state_ids = np.zeros(len(State_Name), dtype=np.int8)
    for i in xrange(0, len(State_Name)):
        state_ids[i] = state_list.index(State_Name[i])
    return state_ids

In [95]:
get_unique_id(np.array([10,10],dtype=np.int8), ['Alabama', 'New Hampshire'], np.array([30.498001, 43.629611]))

[ 0 28]
[49 62]
[10 10]
[       0 28000000]
[ 1000.  1000.]
[49 62]


array([  1.04900000e+03,   2.80010620e+07])

In [56]:
lats = np.array([30.498001, 43.629611])
a = lats*100
(lats*100).astype(int) % (lats.astype(int)*100)

array([49, 62])